In [3]:
import pickle
from datetime import datetime, timedelta, timezone
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import sys

#adds folder to the search path for 'import'
sys.path.append('../python-files')
sys.path.append('../broker')  #this is where client.py lives

import client



In [4]:
results = pickle.load(open('results.pkl', 'rb'))

AttributeError: Can't get attribute 'Utc' on <module 'iso8601.iso8601' from '/home/user/.local/lib/python3.5/site-packages/iso8601/iso8601.py'>

In [0]:
ap = results['Apomahon']
print(ap.name)
print(ap.lat)
print(ap.lon)
print(ap.alt)
#print(ap.calendar)
print(ap.busy_time())

In [0]:
vu = results['Valparaiso University']
print(vu.name)
print(vu.busy_time())

In [0]:
def passlines(ax, y, xstart, xstop, color='b'):
    ax.hlines(y, xstart, xstop, color, lw=4)
    #plt.vlines(xstart, y+0.03, y-0.03, color, lw=2)
    #plt.vlines(xstop, y+0.03, y-0.03, color, lw=2)

In [0]:
x = vu.calendar_begin()
z = vu.calendar_end()

In [0]:
results['Apomahon'].busy_time()

In [0]:
starters = []
enders = []
sats = []
for p in sorted(vu.calendar.search(x, x + timedelta(hours=4))):
    starters.append(p.begin)
    enders.append(p.end)
    sats.append(p.data['job']['tle0'])

In [0]:
from datetime import timedelta
oneday = timedelta(days=1)

In [0]:
import numpy as np

plt.close('all')
fig = plt.figure(figsize=(10, 4))

ax = plt.gca()
ax.xaxis_date()
ax.set_ylim(0.8,2)
#plt.xlim(x,z)
#plt.xlim(x,x+oneday)


#plt.xlim(x, x + timedelta(hours=4))


ax.hlines(np.ones_like(starters), starters, enders, lw=4)
ax.vlines(starters, 1+0.03, 1-0.03, 'r', lw=2)
ax.vlines(enders, 1+0.03, 1-0.03, 'r', lw=2)
for sat, start, end in zip(sats, starters, enders):
    midpoint = start + (end-start)/2
    ax.text(midpoint, 1.1, sat, rotation=90, ha='center', va='bottom')

ax.xaxis.set_major_locator(mdates.HourLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))

ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=range(0, 60, 10)))
#ax.xaxis.set_minor_formatter(mdates.DateFormatter('%H:%M'))
ax.format_xdata = mdates.DateFormatter('%H:%M')

#turn off y numbers
ax.set_yticks([])
fig.autofmt_xdate() #rotates date labels

ax.set_title('Valparaiso University jobs')
plt.show()


In [0]:
def std_calendar_plt(client,hours):
    starters = []
    enders = []
    sats = []
    for p in sorted(client.calendar.search(x, x + timedelta(hours=HOURS))):
        starters.append(p.begin)
        enders.append(p.end)
        sats.append(p.data['job']['tle0'])
    plt.close('all')
    fig = plt.figure(figsize=(25, 4))

    ax = plt.gca()
    ax.xaxis_date()
    ax.set_ylim(0.8,2)

    ax.hlines(np.ones_like(starters), starters, enders, lw=4)
    ax.vlines(starters, 1+0.03, 1-0.03, 'r', lw=2)
    ax.vlines(enders, 1+0.03, 1-0.03, 'r', lw=2)
    for sat, start, end in zip(sats, starters, enders):
        midpoint = start + (end-start)/2
        ax.text(midpoint, 1.1, sat, rotation=90, ha='center', va='bottom')

    ax.xaxis.set_major_locator(mdates.HourLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))

    ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=range(0, 60, 10)))

    ax.format_xdata = mdates.DateFormatter('%H:%M')

    #turn off y numbers
    ax.set_yticks([])
    fig.autofmt_xdate() #rotates date labels

    ax.set_title('%s Jobs' % (c))
    plt.show()


In [0]:
c = results['FACT']
HOURS = 23
std_calendar_plt(c,HOURS)

In [0]:
HOURS = 23
for c, value in results.items():
    std_calendar_plt(results[value.name], HOURS)